This is a notebook for developing energy flux calculations.

Energy Flux:

$\vec{F} = <\vec{u}p> $ where $<>$ means average over wave period.

Then we can get a baroclinic and barotropic energy flux by substituting baroclinc and barotropic expressions for $\vec{u}$ and $p$.

For example, (Cummins and Oey, 1996)

$p_{bt} = \rho_0g\eta$ 

and

$\frac{\partial p_{bc}}{\partial t} = -\rho_0 \int_0^z wN^2 dz^*$.

The baroclinic and barotropic velocities are set up in their usual ways. 

If we have a tidal analysis for the pressure and also tidal ellipses, so that

$maj= \sum_n maj_n\cos(\omega_n t -\phi_{n})$

$min= \sum_n min_n\sin(\omega_n t -\phi_{n})$

$p= \sum_n p_n\cos(\omega_n t -\phi_{pn})$

Then we can have an energy flux calculation for each tidal constituent $(n)$.

So,
$ F_{maj}^n =\frac{1}{2} maj_n p_n \cos(\phi_n-\phi_{pn})$

$ F_{maj}^n =\frac{1}{2} min_n p_n \sin(\phi_n-\phi_{pn})$

And then rotate to x/y with the ellipse inclination $\theta_n$:

$
\begin{pmatrix}
F_x^n \\
F_y^n 
\end{pmatrix}
= 
\begin{pmatrix}
\cos(\theta_n) & -\sin(\theta_n) \\
\sin(\theta_n) & \cos(\theta_n) 
\end{pmatrix}
\begin{pmatrix}
F_{maj}^n \\
F_{min}^n 
\end{pmatrix}
$

At the moment, I have tidal analysis for baroclinic/barotropic $u$ and $v$, $p_{bc}'$ and $\eta$. But I don't have tidal analysis for the full u/v or p. Perhaps this is something to work up to...

Calculations needed still:

1. Calculate $p_{bc}$ by integrating harmonic analysis of $p_{bc}'$. That is, amplitude --> amplitude/omega and pashe ---> phase +90 deg
2. Calculate $p_{bt}$ from haromcinc analysis of $\eta$.
3. Calulate F_maj, F_min. Write a function to take p, maj, min as arguments so that I can do both baroclinic/barotropic with one function.
4. Rotate to x/y. Function should take Fmin, Fmaj, theta as arguments.

In [2]:
import matplotlib.pylab as plt
import numpy as np
import scipy.io as sio
import netCDF4 as nc

import os

%matplotlib inline

In [3]:
rho0 = 1035 #reference density (kg/m^3)
g = 9.81 #acceleration due to gravity (m/s^2)

In [5]:
path = '/ocean/nsoontie/MEOPAR/TidalEllipseData/CODAR/'

data = sio.loadmat(os.path.join(path, 'CODAR_region_20141126_20150426_ssh'))
sshstruc = data['sshstruc']

data = sio.loadmat(os.path.join(path, 'CODAR_region_20141126_20150426_w'))
wstruc = data['wstruc']

data = sio.loadmat(os.path.join(path, 'CODAR_region_20141126_20150426_pbc_t'))
pbc_t_struc = data['pbc_t_struc']

In [6]:
def barotropic_pressure(sshstruc, const):
    """Calculates the amplitude and phase of the barotropi pressure signal for a constituent"""
    
    var = ssstruc[const]

    amp = var[0, 0]['ampl'][0, 0][:]
    amp = np.ma.masked_invalid(amp)

    phase = var[0, 0]['phas'][0, 0][:]
    phase = np.ma.masked_invalid(phase)
    
    amp = amp*g*rho0

    return amp, phase

In [7]:
def cph2rps(freq):
    """Convert frequency in cycles per hours to radians per second"""
    return freq*2*pi/3600

In [9]:
def baroclinic_pressure(pbc_t_struc,const):
    
    var = ssstruc[const]
    # t_tide records frequency in cylces/hour
    # convert to 
    freq = cph2rps(var[0,0]['freq'][0,0])
    
    amp =  var[0, 0]['ampl'][0, 0][:]
    amp = np.ma.masked_invalid(amp)

    phase = var[0, 0]['phas'][0, 0][:]
    phase = np.ma.masked_invalid(phase)
    
    p_amp = amp/freq
    p_phase = phase+90
    
    return p_amp, p_phase